<a href="https://colab.research.google.com/github/HemanthhVV/Apache-Beam/blob/main/Beam_NoteBook_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install apache-beam

In [ ]:
!pip install apache-beam[interactive]

In [ ]:
import apache_beam as beam

## ParDo and Other Functions in Beam

In [ ]:
dir(beam)

In [ ]:
dir(beam.stats)

In [ ]:
class SplitUp(beam.DoFn):
    def process(self,input):
        return input.split() | beam.combiners.Count.Globally()
# def Counter(input):
#     return input | beam.ParDo(SplitUp()) | beam.combiners.Count()
with beam.Pipeline() as p:
    input = p | beam.Create(["How are you bagga yaro?"])
    input | beam.ParDo(SplitUp()) | beam.Map(print)

5


In [ ]:
class Output(beam.PTransform):
    class _OutputFn(beam.DoFn):
        def __init__(self, prefix=''):
            super().__init__()
            self.prefix = prefix

        def process(self, element):
            print(self.prefix+str(element))

    def __init__(self, label=None,prefix=''):
        super().__init__(label)
        self.prefix = prefix

    def expand(self, input):
        input | "Output" >> beam.ParDo(self._OutputFn(self.prefix))


# DoFn with tokenize function
class BreakIntoWordsDoFn(beam.DoFn):
    def process(self, element):
        return element.split() | "Filtered"  >> beam.Filter(lambda word:word.startswith("H")) | "Counted" >> beam.combiners.Count.Globally()


with beam.Pipeline() as p:
  (p | "Creating Data" >> beam.Create(['Hello Beam', 'It is awesome'])
     # Transform with tokenize DoFn operation
     | "Transformation Starts" >> beam.ParDo(BreakIntoWordsDoFn())
     | "Transformation Completes" >> Output())

1


0


In [ ]:
with beam.Pipeline() as p:
    words_with_counts = p | 'Create words with counts' >> beam.Create([
    ('Hello', 1), ('World', 2), ('How', 3), ('are', 4), ('you', 5)])

    split_words = words_with_counts | 'Split words' >> beam.FlatMap(
    lambda word_with_count: [word_with_count[0]] * word_with_count[1])

    split_words | beam.Map(print)

Hello
World
World
How
How
How
are
are
are
are
you
you
you
you
you


In [ ]:
p = split_words | beam.Map(print)

In [ ]:
p

<PCollection[[14]: Map(print).None] at 0x7b9b0fe23a30>

In [ ]:
import datetime

In [ ]:
import pytz

In [ ]:
pytz.utc

<UTC>

In [ ]:
datetime.datetime(2020, 3, 4, 0, 0, 0, 0, tzinfo=pytz.UTC)

datetime.datetime(2020, 3, 4, 0, 0, tzinfo=<UTC>)

In [ ]:
from apache_beam import window


class Output(beam.PTransform):
    class _OutputFn(beam.DoFn):
        def __init__(self, prefix=''):
            super().__init__()
            self.prefix = prefix

        def process(self, element):
            print(self.prefix+str(element))

    def __init__(self, label=None,prefix=''):
        super().__init__(label)
        self.prefix = prefix

    def expand(self, input):
        input | beam.ParDo(self._OutputFn(self.prefix))

class Event:
    def __init__(self, id, event, timestamp):
        self.id = id
        self.event = event
        self.timestamp = timestamp

    def __str__(self) -> str:
        return f'Event({self.id}, {self.event}, {self.timestamp})'


class AddTimestampDoFn(beam.DoFn):
    def process(self, element, **kwargs):
        unix_timestamp = element.timestamp.timestamp()
        yield window.TimestampedValue(element, unix_timestamp)


with beam.Pipeline() as p:
  (p | beam.Create([
          Event('1', 'book-order', datetime.datetime(2020, 3, 4, 0, 0, 0, 0, tzinfo=pytz.UTC)),
          Event('2', 'pencil-order', datetime.datetime(2020, 3, 5, 0, 0, 0, 0, tzinfo=pytz.UTC)),
          Event('3', 'paper-order', datetime.datetime(2020, 3, 6, 0, 0, 0, 0, tzinfo=pytz.UTC)),
          Event('4', 'pencil-order', datetime.datetime(2020, 3, 7, 0, 0, 0, 0, tzinfo=pytz.UTC)),
          Event('5', 'book-order', datetime.datetime(2020, 3, 8, 0, 0, 0, 0, tzinfo=pytz.UTC)),
       ])
     | beam.ParDo(AddTimestampDoFn())
     | 'Log words' >> Output())


Event(1, book-order, 2020-03-04 00:00:00+00:00)
Event(2, pencil-order, 2020-03-05 00:00:00+00:00)
Event(3, paper-order, 2020-03-06 00:00:00+00:00)
Event(4, pencil-order, 2020-03-07 00:00:00+00:00)
Event(5, book-order, 2020-03-08 00:00:00+00:00)
